# UCM Applied Mathematics Department's Problem of the Month

March 2022

Andrew Lin

## The problem

All code and data is found [here](https://github.com/classAndrew/ampom/tree/main/mar2022).

Can you arrange integers 1 to 32 in a circle such that adjacent sums add to a perfect square?

Solution: Here's one of the 64 ways
`1 8 28 21 4 32 17 19 30 6 3 13 12 24 25 11 5 31 18 7 29 20 16 9 27 22 14 2 23 26 10 15`




## The Simpler, Linear Case

The example provided for numbers 1 to 15 is useful:

`8,1,15,10,6,3,13,12,4,5,11,14,2,7,9`

is a sequence such that the problem statement is *almost* satisfied. I will call this **linearly satisfied**.

If a circular arrangement is found, then it is **circularly satisfied**.

### Definitions

Let $\mathbb{P}$ be the set of perfect squares that can be formed from the sum of two numbers from $1$ through $n$.

Let $a = a_0,a_1,a_2,..., a_{n-1} $ be the sequence of $n$ elements such that for $1\le i \le n-1$ and $a_{i-1}+a_i \in \mathbb{P}$.

In other words, $a$ is a sequence that is *linearly satisfied*.

$a$ is *circularly satisfied* when $a_{n-1}+a_0 \in \mathbb{P}$


### Initial Tests

`8,1,15,10,6,3,13,12,4,5,11,14,2,7,9`  is a permutation of  `1,2,3,4,5,6,7,8,9,10,11,12,13,14,15`,

let's see how many iterations a computer would roughly have to do. (Remember, 15! permutations times 15 checks)

In [1]:
from math import factorial; factorial(15)*15

19615115520000

It'll take roughly 6 hours for a computer to find such a permutation by naively bruteforcing, and this is just for 16 elements... so let's apply some critical reasoning to help speed things up.

## Backtracking

Backtracking is a type of optimized bruteforce that is implemented as a recursive algorithm. The most famous example is the N-Queens problem: How many ways can you rearrange N-Queens on an $N \times N$ chess board without any of the queens attacking each other?

The recursive invariant of the N-Queens problem is that only the next queen's placement matters because all the previous queens are assumed to not attack each other.

The recursive invariant of this month's problem is that we only care about the last element of the sequence because all the previous numbers are assumed to be *linearly satisfied*

Take the example:

`8,1,15,6`

It wouldn't make any sense for the next number to be a $2$ because $6+2=8 \not\in \mathbb{P}$. So the next number has to sum to a number in $\mathbb{P}$. $3$ could work as a next choice because $3+6=9 \in \mathbb{P}$. If we don't have any numbers left, we backtrack by removing the last number and trying a different number.

### One more thing...

We still don't know what $\mathbb{P}$ is yet. Let's just assume that it's the range of perfect squares formed by the smallest perfect square of any two ($4$, because $0$ cannot be in the sequence so $1$ is not a possible choice) and the largest perfect square that you can make from two numbers in the sequence (which is the largest square less than $2N-1$)

I get $2N-1$ from adding the largest two in the sequence

So our case for $N=15$,  $\mathbb{P} = \{4, 9, 16, 25\}$

The first element in the sequence can be anything from $1$ to $N$

## Some implementation details

I implemented this algorithm in C++ to be as fast as possible thinking that it'd still take a few hours for the $N=32$ case.

I was completely wrong... the program finished running in a tenth of a second.

### Implementation notes
I used a bitset which is a bithack that determines if some value that can be mapped to an integer is inside a set. Set operations like checking for existence, insertion, and removal are *unbelievably* fast on bitsets. The problem asks for $N=32$ which is just 1 bit from the limit of a 4 byte int!

The base case is hit and a sequence that is *linearly satisfied* is found when $i=N$.

An implicit base case is hit when no numbers are available that add to a perfect square.

Indexing starts at 0, so the index for the bitset must be shifted down one.

```cpp
#include <iostream>

using namespace std;

const uint32_t N = 32;

int sequence[N] = {};
// 0=free, 1=used
uint64_t bitset = 0;

uint32_t P_len = 6;
const int P[] = {4, 9, 16, 25, 36, 49};

void backtrack(int i) {
    if (i == N) {
        // we made it 
        for (int i = 0; i < N; i++) cout << sequence[i] << ' ';
        cout <<'\n';
        return;
    }

    for (int j = 0; j < P_len; j++) {
        int next = P[j]-sequence[i-1];
        // next is taken
        if (next <= 0 || next > N || bitset & (1 << (next-1))) continue;

        sequence[i] = next;
        bitset |= (1 << (next-1));
        backtrack(i+1);
        bitset &= ~(1 << (next-1));
    }

    // there's nothing left to do
}

int main() {
    for (int i = 0; i < N; i++) {
        sequence[0] = i+1;
        bitset |= (1 << i);
        backtrack(1);
        bitset &= ~(1 << i);
    }
}
```

In [4]:
# checking for circularly satisfied
circular = []
P = {4, 9, 16, 25, 36, 49}
with open("linear_square.txt") as f:
   linear = [[int(y) for y in x.split(' ')[:-1]] for x in f.read().split('\n')]
for seq in linear:
    if seq[-1]+seq[0] in P: circular.append(seq)

print('\n'.join(' '.join(map(str, x)) for x in circular))

1 8 28 21 4 32 17 19 30 6 3 13 12 24 25 11 5 31 18 7 29 20 16 9 27 22 14 2 23 26 10 15
1 15 10 26 23 2 14 22 27 9 16 20 29 7 18 31 5 11 25 24 12 13 3 6 30 19 17 32 4 21 28 8
2 14 22 27 9 16 20 29 7 18 31 5 11 25 24 12 13 3 6 30 19 17 32 4 21 28 8 1 15 10 26 23
2 23 26 10 15 1 8 28 21 4 32 17 19 30 6 3 13 12 24 25 11 5 31 18 7 29 20 16 9 27 22 14
3 6 30 19 17 32 4 21 28 8 1 15 10 26 23 2 14 22 27 9 16 20 29 7 18 31 5 11 25 24 12 13
3 13 12 24 25 11 5 31 18 7 29 20 16 9 27 22 14 2 23 26 10 15 1 8 28 21 4 32 17 19 30 6
4 21 28 8 1 15 10 26 23 2 14 22 27 9 16 20 29 7 18 31 5 11 25 24 12 13 3 6 30 19 17 32
4 32 17 19 30 6 3 13 12 24 25 11 5 31 18 7 29 20 16 9 27 22 14 2 23 26 10 15 1 8 28 21
5 11 25 24 12 13 3 6 30 19 17 32 4 21 28 8 1 15 10 26 23 2 14 22 27 9 16 20 29 7 18 31
5 31 18 7 29 20 16 9 27 22 14 2 23 26 10 15 1 8 28 21 4 32 17 19 30 6 3 13 12 24 25 11
6 3 13 12 24 25 11 5 31 18 7 29 20 16 9 27 22 14 2 23 26 10 15 1 8 28 21 4 32 17 19 30
6 30 19 17 32 4 21 28 8 1 15 10 26 23 2 14 

### Results
There are **784** *linearly satisfied* sequences,
**64** of which are *circularly satisfied* sequences.